In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 99

In [2]:
sh = pd.read_csv("../../output/header_sims_info.tsv", sep="\t")
sinfo = pd.read_csv("../../output/sims_info.tsv", sep="\t", header=None, names=sh.columns)
sinfo['date'] = sinfo['date'].str.split('_', expand=True).iloc[:,[1,0]].agg('_'.join, axis=1)
sinfo = sinfo.round(15)
sinfo.rescf = sinfo.rescf.astype(str)
sinfo.sort_values(by=["date"], inplace=True)

In [3]:
n_trees = sinfo.edge.unique().size

In [4]:
sh

,edge,parent,N,gens,rescf,padding,delrate,delcoef,posrate,poscoef,...,rand_id,supnumid,sup_rand_id,outfile_pre,exonfile,recfile,recfilehap,par_string,date,log


In [5]:
sinfo.shape
sinfo.columns

Index(['edge', 'parent', 'N', 'gens', 'rescf', 'padding', 'delrate', 'delcoef',
       'posrate', 'poscoef', 'chr', 'start', 'end', 'win_id', 'win_len',
       'clen', 'padded_start', 'padded_end', 'L', 'mu', 'delprop', 'posprop',
       'siminterval', 'numid', 'rand_id', 'supnumid', 'sup_rand_id',
       'outfile_pre', 'exonfile', 'recfile', 'recfilehap', 'par_string',
       'date', 'log'],
      dtype='object')

In [39]:
diff_cols = ["delrate","delcoef","posrate","poscoef","mu"]
# add poscoefdecayeps

In [40]:
ddf = sinfo[diff_cols].copy()
ddf.drop_duplicates(subset=None, keep="last", inplace=True)

In [43]:
ddf

,delrate,delcoef,posrate,poscoef,mu
16505,6.000000e-09,-0.015,1.000000e-11,0.01,6.010000e-09
16366,6.000000e-09,-0.015,0.000000e+00,0.00,6.000000e-09
16368,0.000000e+00,0.000,1.000000e-11,0.01,1.000000e-11
16421,6.000000e-09,-0.030,1.000000e-10,0.01,6.100000e-09
16439,2.000000e-09,-0.015,1.000000e-11,0.01,2.010000e-09
16436,2.000000e-09,-0.015,1.000000e-10,0.01,2.100000e-09
16409,2.000000e-09,-0.030,1.000000e-10,0.01,2.100000e-09
16408,2.000000e-09,-0.030,1.000000e-11,0.01,2.010000e-09
16415,2.000000e-09,-0.015,0.000000e+00,0.00,2.000000e-09
16399,6.000000e-09,-0.030,0.000000e+00,0.00,6.000000e-09


In [9]:
sup_has_all_branches = (ddf.groupby(by=['sup_rand_id']).size()<14)

In [10]:
g = sinfo.groupby(["rand_id"])
s = g.size()
not_all_edges = s[s<sinfo.edge.unique().size]

In [17]:
import os
root_dir = "../../output/"
not_all_tss = []
not_in_sinfo = []
for file in os.listdir(root_dir):
    fpath = root_dir+file
    if os.path.isdir(fpath):
        is_ts = [f.endswith("trees") for f in os.listdir(fpath)]
        if sum(is_ts) < n_trees:
            not_all_tss.append(file)
        if (sinfo.rand_id == file).sum() == 0:
            not_in_sinfo.append(file)

In [19]:
not_in_sinfo

['figs', 'backup', 'tmp']

In [20]:
import shutil
for i in not_all_tss:
    if not any(sinfo[sinfo.rand_id == i].date>'30022021_0000') and (len(i) >10):
        #print(root_dir+i)
        shutil.rmtree(root_dir+i)
for i in not_in_sinfo:
    if len(i)>10:
        shutil.rmtree(root_dir+i)

In [22]:
(sinfo.rand_id.isin(os.listdir(root_dir))).sum()

19874

In [26]:
sinfo[sinfo.rescf.astype(int)>1]

,edge,parent,N,gens,rescf,padding,delrate,delcoef,posrate,poscoef,...,rand_id,supnumid,sup_rand_id,outfile_pre,exonfile,recfile,recfilehap,par_string,date,log
16,nigerian-chimp,nigerian-western,8800,9400,10,0,2.000000e-09,-0.03,1.000000e-10,0.01,...,87Z02KGAZS9KJB7,39,QILSCCBPY9JM1Y9ZZ0,../../output/nigerian-chimp_87Z02KGAZS9KJB7,../../meta/maps/87Z02KGAZS9KJB7_exons.tsv,../../meta/maps/87Z02KGAZS9KJB7_recrate.tsv,../../meta/maps/87Z02KGAZS9KJB7_recrate.hapmap,"-d recfile=\""../../meta/maps/87Z02KGAZS9KJB7_r...",01022021_0718,../../output/tmp/nigerian-chimp_87Z02KGAZS9KJB...
14,eastern-gorilla,gorilla,4211,7895,10,0,2.000000e-09,-0.03,1.000000e-11,0.01,...,ZGNJ76X91BIHQKI,35,562V5VDT6Y66UT3SKV,../../output/eastern-gorilla_ZGNJ76X91BIHQKI,../../meta/maps/ZGNJ76X91BIHQKI_exons.tsv,../../meta/maps/ZGNJ76X91BIHQKI_recrate.tsv,../../meta/maps/ZGNJ76X91BIHQKI_recrate.hapmap,"-d recfile=\""../../meta/maps/ZGNJ76X91BIHQKI_r...",01022021_0718,../../output/tmp/eastern-gorilla_ZGNJ76X91BIHQ...
22,nigerian-chimp,nigerian-western,8800,9400,10,0,2.000000e-09,-0.03,1.000000e-10,0.01,...,OI6PO7PFQJ8OXWG,39,QILSCCBPY9JM1Y9ZZ0,../../output/nigerian-chimp_OI6PO7PFQJ8OXWG,../../meta/maps/OI6PO7PFQJ8OXWG_exons.tsv,../../meta/maps/OI6PO7PFQJ8OXWG_recrate.tsv,../../meta/maps/OI6PO7PFQJ8OXWG_recrate.hapmap,"-d recfile=\""../../meta/maps/OI6PO7PFQJ8OXWG_r...",01022021_0718,../../output/tmp/nigerian-chimp_OI6PO7PFQJ8OXW...
9,nigerian-western,chimps,10100,7720,10,0,0.000000e+00,0.00,1.000000e-10,0.01,...,CV84TR0YCECX4KD,59,5QXE1D7WE2XZIALIV8,../../output/nigerian-western_CV84TR0YCECX4KD,../../meta/maps/CV84TR0YCECX4KD_exons.tsv,../../meta/maps/CV84TR0YCECX4KD_recrate.tsv,../../meta/maps/CV84TR0YCECX4KD_recrate.hapmap,"-d recfile=\""../../meta/maps/CV84TR0YCECX4KD_r...",01022021_0718,../../output/tmp/nigerian-western_CV84TR0YCECX...
7,orangutans,great-apes,19231,409731,10,500000,2.000000e-09,-0.03,1.000000e-11,0.01,...,IXVY6VGC98JZWBI,65,BUIGSIOWMVJDIROAC9,../../output/orangutans_IXVY6VGC98JZWBI,../../meta/maps/IXVY6VGC98JZWBI_exons.tsv,../../meta/maps/IXVY6VGC98JZWBI_recrate.tsv,../../meta/maps/IXVY6VGC98JZWBI_recrate.hapmap,"-d recfile=\""../../meta/maps/IXVY6VGC98JZWBI_r...",01022021_0718,../../output/tmp/orangutans_IXVY6VGC98JZWBI.log
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16551,eastern-chimp,eastern-central,12400,7000,100,500000,2.000000e-09,-0.03,0.000000e+00,0.00,...,TZPNGS0UY29NGB3,1,BLRVBDHFBEM6WYNNOX,../../output/eastern-chimp_TZPNGS0UY29NGB3,../../meta/maps/TZPNGS0UY29NGB3_exons.tsv,../../meta/maps/TZPNGS0UY29NGB3_recrate.tsv,../../meta/maps/TZPNGS0UY29NGB3_recrate.hapmap,"-d recfile=\""../../meta/maps/TZPNGS0UY29NGB3_r...",01022021_0911,../../output/tmp/eastern-chimp_TZPNGS0UY29NGB3...
16550,african-apes,great-apes,47059,275950,100,500000,2.000000e-09,-0.03,0.000000e+00,0.00,...,TZPNGS0UY29NGB3,1,BLRVBDHFBEM6WYNNOX,../../output/african-apes_TZPNGS0UY29NGB3,../../meta/maps/TZPNGS0UY29NGB3_exons.tsv,../../meta/maps/TZPNGS0UY29NGB3_recrate.tsv,../../meta/maps/TZPNGS0UY29NGB3_recrate.hapmap,"-d recfile=\""../../meta/maps/TZPNGS0UY29NGB3_r...",01022021_0911,../../output/tmp/african-apes_TZPNGS0UY29NGB3.log
16549,humans,human-pan,7672,129828,100,500000,2.000000e-09,-0.03,0.000000e+00,0.00,...,TZPNGS0UY29NGB3,1,BLRVBDHFBEM6WYNNOX,../../output/humans_TZPNGS0UY29NGB3,../../meta/maps/TZPNGS0UY29NGB3_exons.tsv,../../meta/maps/TZPNGS0UY29NGB3_recrate.tsv,../../meta/maps/TZPNGS0UY29NGB3_recrate.hapmap,"-d recfile=\""../../meta/maps/TZPNGS0UY29NGB3_r...",01022021_0911,../../output/tmp/humans_TZPNGS0UY29NGB3.log
16556,nigerian-western,chimps,10100,7720,100,0,2.000000e-09,-0.03,0.000000e+00,0.00,...,Z6BZODZEHIX881D,0,ZVHXSOLX8VS317Q1CD,../../output/nigerian-western_Z6BZODZEHIX881D,../../meta/maps/Z6BZODZEHIX881D_exons.tsv,../../meta/maps/Z6BZODZEHIX881D_recrate.tsv,../../meta/maps/Z6BZODZEHIX881D_recrate.hapmap,"-d recfile=\""../../meta/maps/Z6BZODZEHIX881D_r...",01022021_0911,../../output/tmp/nigerian-western